In [1]:
from PIL import Image
from safetensors import safe_open
from torchvision import transforms
from transformers import (AutoConfig, AutoProcessor, AutoTokenizer, AutoModelForVision2Seq)
import torch, os, requests
from io import BytesIO
from transformers.dynamic_module_utils import get_class_from_dynamic_module

2024-08-13 16:09:12.194679: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-13 16:09:12.216122: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 16:09:12.216143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 16:09:12.217069: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-13 16:09:12.221244: I tensorflow/core/platform/cpu_feature_guar

In [2]:
device = "cuda"

In [8]:
pretrained_model_name_or_path = "Embodied-CoT/ecot-openvla-7b-bridge"
config = AutoConfig.from_pretrained(
                pretrained_model_name_or_path,
                trust_remote_code=True
            )
vision_backbone_class_ref = config.auto_map[AutoModelForVision2Seq.__name__].replace("OpenVLAForActionPrediction", "PrismaticVisionBackbone")
vision_backbone_class = get_class_from_dynamic_module(
    vision_backbone_class_ref, pretrained_model_name_or_path
)
vision_backbone = vision_backbone_class(
            config.use_fused_vision_backbone, config.image_sizes, config.timm_model_ids, config.timm_override_act_layers
        )
vision_backbone.load_state_dict(torch.load("../../TensorRT-LLM/save_dir/vision_backbone.pth", map_location=device))
vision_backbone.featurizer.to(device, torch.bfloat16)

/home/will/miniconda3/envs/tensorrt-ecot/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1152, out_features=3456, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1152, out_features=1152, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1152, out_features=4304, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Id

In [17]:
proj_class_ref = config.auto_map[AutoModelForVision2Seq.__name__].replace("OpenVLAForActionPrediction", "PrismaticProjector")
proj_class = get_class_from_dynamic_module(
    proj_class_ref, pretrained_model_name_or_path
)
projector = proj_class(
    config.use_fused_vision_backbone,
    vision_dim=vision_backbone.embed_dim,
    llm_dim=config.text_config.hidden_size,
)
projector.load_state_dict(torch.load("../../TensorRT-LLM/save_dir/projector.pth", map_location=device))
projector.to(device, torch.bfloat16)

/home/will/miniconda3/envs/tensorrt-ecot/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


PrismaticProjector(
  (fc1): Linear(in_features=1152, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (act_fn1): GELU(approximate='none')
)

In [5]:
url = 'https://raw.githubusercontent.com/MichalZawalski/embodied-CoT/main/test_obs.png'
page = requests.get(url)
image = Image.open(BytesIO(page.content))

In [6]:
processor = AutoProcessor.from_pretrained(pretrained_model_name_or_path, trust_remote_code=True)

In [10]:
pixel_values = processor("", image)["pixel_values"].to(device, dtype=torch.bfloat16)
patch_features = vision_backbone(pixel_values)

In [18]:
projector(patch_features)

tensor([[[ 0.0388, -0.0889,  0.4023,  ...,  0.3066,  0.5820, -0.2168],
         [-0.2227,  0.6016, -0.2480,  ...,  0.2070,  0.3965, -0.1270],
         [ 0.0544, -0.0452, -0.2207,  ...,  0.0110,  0.1543,  0.1021],
         ...,
         [ 0.0145,  0.3164, -0.2188,  ...,  0.2598,  0.2656, -0.2422],
         [ 0.0309,  0.0023,  0.1436,  ...,  0.0317,  0.4961, -0.2812],
         [ 0.2080, -0.2041,  0.3691,  ...,  0.2520,  0.5078, -0.3770]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<ViewBackward0>)